### rackcli - h4x VCV Rack via command line

This is a collection of research discussions/prototypes for icubating ideas and features for [VCV Rack](https://vcvrack.com/) under the umbrella of [rackcli](https://github.com/dirkleas/rackcli). These presentations assume metadata extracted via my [VCV Rack fork](https://github.com/dirkleas/Rack) or using my plugin [DLwigglz r4kH4x](https://github.com/dirkleas/DLwigglz) along with an upcoming webservice [r4xH4x.cloud]() accessed through `rackcli`.  There's also series of companion 
[videos](https://www.youtube.com/channel/UCv-mq6lyycCbvbQiZclik7Q)!

Over time, this collection will grow to support a number of areas, ranging from simple ideas like patch module slicing through more sophisticated ideas like plugin/module analytics and patch construction using machine learning. Research such as this is plausible give that VCV Rack patches are stored as JSON text files -- the only "extras" are adding some extra metadata for things like module widths and a reference catalog of all your installed plugin modules (of course that includes width values too).

That last little requirement for the extra metadata (e.g. width) is most easily accomplished usig a [fork I made](https://github.com/dirkleas/Rack) of the official VCV Rack project as it introduced a few minor changes to augment the existing patch save functionality width support, and a new function accessible via the `File.Catalog` menu to create a JSON catalog for all your installed plugins and their modules. This file is stored in your forked VCV Rack build folder. Another option not requiring using a fork is to use my plugin [DLwigglz r4xH4x](https://github.com/dirkleas/DLwigglz) module and `rackcli` to fill in the metadata holes. Here's an example fragment from the `catalog.json` recently generated from my installed plugins sourced from the Plugin Manager, and and other online sources (this is a very small fragment as I've typically got about 100 plugins installed) -- notice the `"width": 195` for the *AnimatedCircuits_Noises ACNoises* below:

```
{
  "applicationName": "VCV Rack",
  "applicationVersion": "0.6.1",
  "apiHost": "https://api.vcvrack.com",
  "width": 1440,
  "height": 878,
  "token": "",
  "path": "/Users/dirkleas/Desktop/h4x/_queue_/vcv/Rack",
  "pluginCount": 91,
  "moduleCount": 2,
  "plugins": [

...

    {
      "slug": "AnimatedCircuits_Noises",
      "path": "./plugins/AnimatedCircuits_Noises",
      "version": "0.6.0",
      "modelCount": 1,
      "models": [
        {
          "slug": "ACNoises",
          "name": "Noises",
          "author": "Animated Circuits",
          "tags": [
            "Noise",
            "Sample and Hold",
            "Slew Limiter",
            "Random",
            "LFO"
          ],
          "width": 195
        }
      ]
    }
  ]
}
```

Building or installing a VCV Rack fork is hard and will mess up my workflow, right?! No, not really. Quite of few rackheads have built a local fork of VCV Rack so they can compile plugin modules from source code when they're either not through the official Plugin Manager workflow, or they're distributed directly by the developers (typically via [Github](https://github.com/topics/vcvrack).

There are two options for you to live happily with my VCV Rack fork: 1) use VCV Rack fork on an as-needed basis, or 2) use the fork most of the time and VCV Rack when you wish to check the Package Manager for updates or new plugins. Here's how each of those workflows might work for you.

##### VCV Rack Fork Centric

1. one time only: create symbolic links from your VCV Rack plugins folder items in your VCV Rack Fork plugins folder -- this will allow you to use all your plugins from either VCV Rack or VCV Rack Fork
1. if you wish to check for Plugin Manager updates, run VCV Rack and optionally update your plugins via `Update plugins` button and automatically exit
1. run VCV Rack Fork as usual and enjoy your fresh `catalog.json`


<img src="https://user-images.githubusercontent.com/52076/39953295-911a6a5c-5576-11e8-965b-0f1743ff2cbd.png" alt="DLwigglz h4xH4x" style="width: 15%; height: 15%; padding: 0 35px; float: right;"/>

##### VCV Rack Centric ("don't need no stink'n forkz!")

1. run VCV Rack and optionally update your plugins via `Update plugins` button and automatically exit
1. if you update your plugins, add the module `r4xH4x` from the plugin `DLwigglz` and click the `Catalog (partial)` button
1. run `rackcli --cloud catalog.partial.json` to load missing metadata -- you'll get feedback in the VCV Rack log `log.txt` identifying any plugins that haven't been cataloged by the community yet, **CURRENTLY WORK-IN-PROCESS!**
1. run VCV Rack as usual and enjoy your fresh `catalog.json`

See, either option adds minimal overhead to your patching workflow, but allows you to take advantage of the extra metadata for exciting reasons you'll soon appreciate. Since the fork is based on VCV Rack, all your patches should work the same -- though there are some additional usability features in the fork that provide a smoother, more flexible patchig workflow.

---

*Now for the pseudo-warning: YMMV! I've found both my fork and plugin services are reasonably stable with plugins from the offical VCV Rack Plugin Manager, but sometimes I have issues with certain modules cleaning up after themselves (on my Mac, that's some of the fancier modules doing cool stuff dealing with MIDI for example). This even more common with bleeding-edge modules that I compile myself. As I identify such plugin modules, I'll just drag their plugin out of my `plugins` folder, and only install it when I need it. I find that when I start automating things/hacking my catalog of plugin modules, I see more because I use more. You get the picture. Just be aware, and enjoy your newfound superpowers!*

---


PS: if using [Jupyter](http://jupyter.org/) notebooks and data science/machine learning platforms like [Anaconda](https://www.anaconda.com/download/#macos) are new to you, there's tons of great learning materials out there -- dig in and enjoy your new superpowers!

Here's an example of what the `catalog.json` and augmented patch files might look like. Sample output has been truncated for covenience -- this was generated from my plugins and I've purchased almost every official module and build/installed many others, last count, almost 100 plugins!

In [1]:
%run rackcli_core.ipynb


rackcli_core imports:
    hp(pixels) - convert pixels to hp
    is_running('rack') - predicate for OS-level process running, default "rack"



In [2]:
from pprint import pprint as pp
from typing import *
from itertools import *
from time import sleep
import json, copy, mido

In [3]:
rackdir = '/Users/dirkleas/Desktop/h4x/_queue_/vcv/Rack' # update to reference your VCV Rack folder
catalog = json.load(open(rackdir + '/catalog.json')) # generated via File.Catalog

def plugin_module(pslug: str, version: str, mslug: str, plugins: Dict) -> Dict:
    'get plugin module or None if non-existent, no version needed?'
    try:
        p = list(filter(lambda p: p['slug'] == pslug and p['version'] == version, plugins))[0]
        m = list(filter(lambda m: m['slug'] == mslug, p['models']))[0]
        return({**m, **{'plugin': p['slug'], 'version': p['version']}})
    except: return(None)

# plugin_module('Core', '0.6.1', 'Notes', catalog['plugins'])
# patch
# catalog

What's something useful we can do with this catalog data? How generating documentationn patches for all your plugins so we can see every single module you've got installed?!?! You'll notice that the width values stored in the catalog are in pixels and the position data stored in patches is in [HP](https://en.wikipedia.org/wiki/Horizontal_pitch) (horizontal pitch), so we'll want to do some simple math to translate back and forth when doing layout stuff. For fun, I'll automate iterating through the generated plugin module patches using MIDI and the awesome keyboard/mouse automation software [Keyboard Maestro](https://www.keyboardmaestro.com/main/) for MacOS (similar tools exist for Windows and Linux, just look for keyboard support.

---

*Warning (again), this will iterate through EVERY plugin module, so there might be some modules that crash VCV Rack that you've never actually loaded before -- just restart Rack and continue if this happens (don't forget to report bugs to the offending module developer so they can fix any issues).*

In [4]:
_rails_hp = hp(catalog['width'])
_patch = {'version': catalog['applicationVersion'], 'modules': [], 'wires': []}
_module = {'plugin': '', 'version': '', 'model': '', 'pos': [0, 0], 'params': [], 'data': {}} # pos[col,row]
_note = {"plugin": "Core", "version": plugin_module('Core', '0.6.1', 'Notes', catalog['plugins'])['version'], 
         "model": "Notes", "pos": [0, 0], "width": 240, "params": [], "text": ""}

def dox(plugins: List, limit: int, skinny: bool=True, pluginFilter: Callable[[Dict, ], bool]=None, 
        modelFilter: Callable[[Dict, ], bool]=None) -> Dict:
    'generate dox patch from specified list of plugins [(pslug, pversion, mslug), ]'
    row = 0; col = 0; d = copy.deepcopy(_patch)
    for p in plugins:
        if not p: continue # ill-formed/incomplete plugin
        if pluginFilter and not pluginFilter(p): continue
        note = copy.deepcopy(_note) # throw in a Core Notes module as table-of-contents
        note['pos'][1] = row
        note['text'] = f"{p['slug']}, v{p['version']}\n\n {p['modelCount']} modules\n\n"
        note['text'] += ''.join([m['name'] + ', ' for m in p['models']])[:-2]
        d['modules'].append(note)
        col += hp(note['width'])
        for m in p['models']:
            if modelFilter and not modelFilter(m): continue
            if skinny and (col + hp(m['width']) >= _rails_hp): row += 1; col = 5 # skinny wrap
            module = copy.deepcopy(_module)
            module['plugin'] = p['slug']; module['version'] = p['version']
            module['model'] = m['slug']; module['pos'] = [col, row]; module['width'] = m['width']
            d['modules'].append(module)
            col += hp(m['width'])
        row += 1; col = 0
    open(rackdir + '/dox.vcv', 'w').write(json.dumps(d))
    return(d)

def chunks(iterable: List, n: int, fillvalue: bool=None) -> List[List[Dict]]:
    'chunk an iterable into n chunks with optional filler'
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def doxer(chunk: int=5, pause: int=10) -> None:
    'iterate through catalog in chunks with delay in seconds with midi triggering refreshes'
    try:
        print('File.reOpen in my VCV Rack fork or File.Revert in VCV Rack to view latest dox ' +
              'when you hear system "bell" ding,\n  ESC+i+i to cancel from your Jupyter notebook...')
        sleep(pause)
        for c in chunks(catalog['plugins'], chunk): # dox plugins a few at a time, adjust per your system specs
            dox(c, 200, True);
            mido.open_output().send(mido.Message('note_on', note=69))
            sleep(pause)
    except KeyboardInterrupt: pass
    print('all done...')

# doxer(pause=7) # auto-nav, ESC+i+i to cancel early
# dox(catalog['plugins'], 200, modelFilter=lambda m: 'Sequencer' in m['tags']) # tag predicate
# dox([p for p in catalog['plugins'] if p['slug'][:4] in ('mtsc', 'Erra', 'moDl')], 200, True)